# 🖼️ Notebook 8: Obrazová Klasifikace s Vision Transformers

**Autor:** Praut s.r.o. - AI Integration & Business Automation

V tomto notebooku se naučíme používat Vision Transformers (ViT) a další modely pro klasifikaci obrázků. Ukážeme si praktické aplikace jako třídění produktových fotografií, kontrolu kvality a automatickou kategorizaci.

## Co se naučíte:
- Základy Vision Transformers
- Klasifikace obrázků pomocí pipeline
- Zero-shot klasifikace s CLIP
- Automatické třídění produktů
- Kontrola kvality obrázků
- Batch zpracování galerie

In [ ]:
# Instalace potřebných knihoven
!pip install -q transformers accelerate torch torchvision pillow requests matplotlib

In [ ]:
import torch
from transformers import pipeline, AutoImageProcessor, AutoModelForImageClassification
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Detekce zařízení
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Používám zařízení: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Pomocné funkce pro práci s obrázky

In [ ]:
def load_image(source):
    """
    Načte obrázek z URL nebo lokální cesty.
    """
    if source.startswith('http'):
        response = requests.get(source, timeout=10)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(source)
    
    # Konverze na RGB pokud je potřeba
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    return image

def show_image_with_results(image, results, title="Výsledky klasifikace"):
    """
    Zobrazí obrázek s výsledky klasifikace.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Obrázek
    ax1.imshow(image)
    ax1.axis('off')
    ax1.set_title('Vstupní obrázek')
    
    # Výsledky
    if isinstance(results, list) and len(results) > 0:
        if isinstance(results[0], dict):
            labels = [r.get('label', r.get('name', 'Unknown'))[:30] for r in results[:5]]
            scores = [r.get('score', 0) for r in results[:5]]
        else:
            labels = [str(r)[:30] for r in results[:5]]
            scores = [1.0] * len(labels)
    else:
        labels = ['Žádné výsledky']
        scores = [0]
    
    colors = plt.cm.viridis([s for s in scores])
    bars = ax2.barh(labels, scores, color=colors)
    ax2.set_xlim(0, 1)
    ax2.set_xlabel('Confidence')
    ax2.set_title(title)
    
    for bar, score in zip(bars, scores):
        ax2.text(score + 0.02, bar.get_y() + bar.get_height()/2, 
                f'{score:.1%}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Test
print("✅ Pomocné funkce připraveny")

## 2. Základní klasifikace s Vision Transformer (ViT)

In [ ]:
# Inicializace pipeline pro klasifikaci obrázků
# ViT natrénovaný na ImageNet-21k
classifier = pipeline(
    "image-classification",
    model="google/vit-base-patch16-224",
    device=0 if device == "cuda" else -1
)

print("✅ Vision Transformer model načten")

In [ ]:
# Test na ukázkovém obrázku
test_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"

image = load_image(test_url)
results = classifier(image, top_k=5)

print("Top 5 predikcí:")
for r in results:
    print(f"  {r['label']}: {r['score']:.1%}")

show_image_with_results(image, results, "ViT Klasifikace")

## 3. Zero-Shot klasifikace s CLIP

CLIP (Contrastive Language-Image Pre-training) umožňuje klasifikovat obrázky podle libovolných textových popisů bez nutnosti dodatečného trénování.

In [ ]:
# Načtení CLIP modelu
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

if device == "cuda":
    clip_model = clip_model.to(device)

print("✅ CLIP model načten")

In [ ]:
def clip_zero_shot_classify(image, candidate_labels):
    """
    Zero-shot klasifikace obrázku pomocí CLIP.
    """
    # Příprava vstupů
    inputs = clip_processor(
        text=candidate_labels,
        images=image,
        return_tensors="pt",
        padding=True
    )
    
    if device == "cuda":
        inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Inference
    with torch.no_grad():
        outputs = clip_model(**inputs)
    
    # Výpočet pravděpodobností
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]
    
    # Seřazení výsledků
    results = [
        {"label": label, "score": float(score)}
        for label, score in zip(candidate_labels, probs)
    ]
    results.sort(key=lambda x: x["score"], reverse=True)
    
    return results

# Test - vlastní kategorie
image = load_image(test_url)
labels = ["a photo of a cat", "a photo of a dog", "a photo of a bird", "a photo of a car"]

results = clip_zero_shot_classify(image, labels)
print("CLIP Zero-Shot výsledky:")
for r in results:
    print(f"  {r['label']}: {r['score']:.1%}")

In [ ]:
# Praktický příklad - třídění produktových fotografií
product_categories = [
    "a product photo of electronics",
    "a product photo of clothing",
    "a product photo of furniture",
    "a product photo of food",
    "a product photo of toys",
    "a product photo of books",
    "a product photo of jewelry",
    "a product photo of sports equipment"
]

# Test s obrázkem notebooku
laptop_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Laptop-matt.svg/1200px-Laptop-matt.svg.png"

try:
    laptop_image = load_image(laptop_url)
    results = clip_zero_shot_classify(laptop_image, product_categories)
    
    print("Klasifikace produktu:")
    for r in results[:3]:
        category = r['label'].replace('a product photo of ', '').title()
        print(f"  {category}: {r['score']:.1%}")
    
    show_image_with_results(laptop_image, results, "Produktová klasifikace")
except Exception as e:
    print(f"Chyba při načítání obrázku: {e}")

## 4. Kontrola kvality obrázků

In [ ]:
class ImageQualityChecker:
    """
    Kontrola kvality obrázků pro e-commerce a marketing.
    """
    
    def __init__(self):
        self.clip_model = clip_model
        self.clip_processor = clip_processor
        
        # Kritéria kvality
        self.quality_labels = [
            "a high quality professional photo",
            "a low quality amateur photo",
            "a blurry out of focus photo",
            "a well-lit bright photo",
            "a dark underexposed photo"
        ]
        
        self.content_labels = [
            "a photo with clear main subject",
            "a cluttered photo with many objects",
            "a photo with clean background",
            "a photo with distracting background"
        ]
    
    def check_quality(self, image):
        """
        Vyhodnotí kvalitu obrázku.
        """
        # Kontrola kvality
        quality_results = clip_zero_shot_classify(image, self.quality_labels)
        
        # Kontrola obsahu
        content_results = clip_zero_shot_classify(image, self.content_labels)
        
        # Výpočet skóre
        quality_score = 0
        issues = []
        
        # Profesionalita
        pro_score = next((r['score'] for r in quality_results if 'professional' in r['label']), 0)
        quality_score += pro_score * 30
        
        # Ostrost
        blur_score = next((r['score'] for r in quality_results if 'blurry' in r['label']), 0)
        if blur_score > 0.3:
            issues.append("Obrázek může být rozmazaný")
        quality_score += (1 - blur_score) * 25
        
        # Osvětlení
        dark_score = next((r['score'] for r in quality_results if 'dark' in r['label']), 0)
        if dark_score > 0.3:
            issues.append("Obrázek může být podexponovaný")
        quality_score += (1 - dark_score) * 25
        
        # Čistota kompozice
        clean_score = next((r['score'] for r in content_results if 'clean background' in r['label']), 0)
        quality_score += clean_score * 20
        
        # Rozlišení
        width, height = image.size
        if width < 800 or height < 600:
            issues.append(f"Nízké rozlišení ({width}x{height})")
            quality_score -= 10
        
        # Výsledek
        quality_score = max(0, min(100, quality_score))
        
        if quality_score >= 80:
            rating = "Výborná"
        elif quality_score >= 60:
            rating = "Dobrá"
        elif quality_score >= 40:
            rating = "Průměrná"
        else:
            rating = "Nízká"
        
        return {
            "score": quality_score,
            "rating": rating,
            "issues": issues,
            "resolution": f"{width}x{height}",
            "details": {
                "professional": pro_score,
                "sharpness": 1 - blur_score,
                "lighting": 1 - dark_score,
                "composition": clean_score
            }
        }

# Test
checker = ImageQualityChecker()
image = load_image(test_url)
quality = checker.check_quality(image)

print("Analýza kvality obrázku:")
print(f"  Celkové skóre: {quality['score']:.0f}/100")
print(f"  Hodnocení: {quality['rating']}")
print(f"  Rozlišení: {quality['resolution']}")
print("  Detaily:")
for key, value in quality['details'].items():
    print(f"    - {key}: {value:.1%}")
if quality['issues']:
    print("  Problémy:")
    for issue in quality['issues']:
        print(f"    ⚠️ {issue}")

## 5. Automatické třídění produktových fotografií

In [ ]:
class ProductImageSorter:
    """
    Automatické třídění produktových fotografií do kategorií.
    """
    
    def __init__(self, categories: dict):
        """
        Args:
            categories: Slovník {název_kategorie: [klíčová_slova]}
        """
        self.categories = categories
        self.quality_checker = ImageQualityChecker()
        
        # Vytvoření CLIP labels
        self.clip_labels = []
        self.label_to_category = {}
        
        for category, keywords in categories.items():
            for keyword in keywords:
                label = f"a photo of {keyword}"
                self.clip_labels.append(label)
                self.label_to_category[label] = category
    
    def classify(self, image, min_confidence=0.3):
        """
        Klasifikuje obrázek do kategorie.
        """
        results = clip_zero_shot_classify(image, self.clip_labels)
        
        # Agregace skóre podle kategorií
        category_scores = {}
        for result in results:
            category = self.label_to_category[result['label']]
            if category not in category_scores:
                category_scores[category] = []
            category_scores[category].append(result['score'])
        
        # Průměrné skóre pro každou kategorii
        final_scores = [
            {"category": cat, "score": sum(scores) / len(scores)}
            for cat, scores in category_scores.items()
        ]
        final_scores.sort(key=lambda x: x['score'], reverse=True)
        
        # Nejlepší kategorie
        best = final_scores[0]
        
        return {
            "category": best['category'] if best['score'] >= min_confidence else "uncategorized",
            "confidence": best['score'],
            "all_scores": final_scores
        }
    
    def process_batch(self, image_sources):
        """
        Zpracuje dávku obrázků.
        """
        results = []
        
        for source in image_sources:
            try:
                image = load_image(source)
                classification = self.classify(image)
                quality = self.quality_checker.check_quality(image)
                
                results.append({
                    "source": source,
                    "category": classification['category'],
                    "confidence": classification['confidence'],
                    "quality_score": quality['score'],
                    "quality_rating": quality['rating'],
                    "issues": quality['issues'],
                    "status": "success"
                })
            except Exception as e:
                results.append({
                    "source": source,
                    "status": "error",
                    "error": str(e)
                })
        
        return results

# Definice kategorií pro e-shop
eshop_categories = {
    "elektronika": ["laptop", "smartphone", "tablet", "headphones", "camera"],
    "obleceni": ["shirt", "pants", "dress", "jacket", "shoes"],
    "nabytek": ["chair", "table", "sofa", "bed", "desk"],
    "domacnost": ["kitchen appliance", "lamp", "decoration", "plant"],
    "sport": ["bicycle", "ball", "fitness equipment", "sports gear"]
}

sorter = ProductImageSorter(eshop_categories)
print("✅ ProductImageSorter připraven")
print(f"   Počet kategorií: {len(eshop_categories)}")

In [ ]:
# Test třídění
test_images = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Golde33443.jpg/640px-Golde33443.jpg",  # pes
    test_url  # kočka
]

for url in test_images:
    try:
        image = load_image(url)
        result = sorter.classify(image)
        
        print(f"\nObrázek: {url[:50]}...")
        print(f"  Kategorie: {result['category']}")
        print(f"  Confidence: {result['confidence']:.1%}")
        print("  Všechny skóre:")
        for score in result['all_scores'][:3]:
            print(f"    - {score['category']}: {score['score']:.1%}")
    except Exception as e:
        print(f"Chyba: {e}")

## 6. Detekce typu obrázku (foto vs. ilustrace vs. screenshot)

In [ ]:
def detect_image_type(image):
    """
    Detekuje typ obrázku.
    """
    type_labels = [
        "a photograph taken with a camera",
        "a digital illustration or drawing",
        "a screenshot from a computer or phone",
        "a 3D render or CGI image",
        "a diagram or chart",
        "a logo or icon",
        "a scanned document"
    ]
    
    results = clip_zero_shot_classify(image, type_labels)
    
    # Mapování na krátké názvy
    type_map = {
        "photograph": "fotografie",
        "illustration": "ilustrace",
        "screenshot": "screenshot",
        "3D render": "3D render",
        "diagram": "diagram",
        "logo": "logo",
        "scanned": "sken dokumentu"
    }
    
    best = results[0]
    detected_type = "neznámý"
    for key, value in type_map.items():
        if key in best['label']:
            detected_type = value
            break
    
    return {
        "type": detected_type,
        "confidence": best['score'],
        "all_types": results
    }

# Test
image = load_image(test_url)
img_type = detect_image_type(image)

print(f"Typ obrázku: {img_type['type']}")
print(f"Confidence: {img_type['confidence']:.1%}")

## 7. Praktická automatizace: E-shop Image Processor

In [ ]:
class EshopImageProcessor:
    """
    Komplexní procesor produktových obrázků pro e-shop.
    """
    
    def __init__(self, categories: dict, min_quality_score=50):
        self.sorter = ProductImageSorter(categories)
        self.min_quality_score = min_quality_score
        
    def process_product_image(self, image_source, product_name=None):
        """
        Kompletní zpracování produktového obrázku.
        """
        try:
            image = load_image(image_source)
        except Exception as e:
            return {
                "status": "error",
                "error": f"Nelze načíst obrázek: {e}"
            }
        
        # 1. Kontrola kvality
        quality = self.sorter.quality_checker.check_quality(image)
        
        # 2. Detekce typu
        img_type = detect_image_type(image)
        
        # 3. Klasifikace kategorie
        classification = self.sorter.classify(image)
        
        # 4. Vyhodnocení vhodnosti
        is_suitable = (
            quality['score'] >= self.min_quality_score and
            img_type['type'] in ['fotografie', '3D render'] and
            classification['confidence'] >= 0.2
        )
        
        # 5. Doporučení
        recommendations = []
        if quality['score'] < self.min_quality_score:
            recommendations.append("Zvažte pořízení kvalitnější fotografie")
        if quality['issues']:
            for issue in quality['issues']:
                recommendations.append(f"Opravte: {issue}")
        if img_type['type'] not in ['fotografie', '3D render']:
            recommendations.append("Pro produkt použijte fotografii nebo 3D render")
        if classification['confidence'] < 0.2:
            recommendations.append("Obrázek neodpovídá žádné kategorii")
        
        return {
            "status": "success",
            "product_name": product_name,
            "is_suitable": is_suitable,
            "quality": {
                "score": quality['score'],
                "rating": quality['rating']
            },
            "image_type": img_type['type'],
            "suggested_category": classification['category'],
            "category_confidence": classification['confidence'],
            "recommendations": recommendations,
            "resolution": quality['resolution']
        }
    
    def batch_process(self, items):
        """
        Dávkové zpracování.
        
        Args:
            items: Seznam slovníků {"source": url/path, "name": název}
        """
        results = []
        stats = {"total": 0, "suitable": 0, "unsuitable": 0, "errors": 0}
        
        for item in items:
            source = item.get("source")
            name = item.get("name", "Unknown")
            
            result = self.process_product_image(source, name)
            results.append(result)
            
            stats["total"] += 1
            if result["status"] == "error":
                stats["errors"] += 1
            elif result["is_suitable"]:
                stats["suitable"] += 1
            else:
                stats["unsuitable"] += 1
        
        return {
            "results": results,
            "statistics": stats
        }

# Inicializace
processor = EshopImageProcessor(eshop_categories, min_quality_score=40)
print("✅ EshopImageProcessor připraven")

In [ ]:
# Test kompletního zpracování
result = processor.process_product_image(test_url, "Testovací produkt")

print("=" * 50)
print("VÝSLEDEK ZPRACOVÁNÍ OBRÁZKU")
print("=" * 50)
print(f"Produkt: {result['product_name']}")
print(f"Status: {result['status']}")
print(f"Vhodný pro e-shop: {'✅ Ano' if result['is_suitable'] else '❌ Ne'}")
print(f"\nKvalita: {result['quality']['score']:.0f}/100 ({result['quality']['rating']})")
print(f"Rozlišení: {result['resolution']}")
print(f"Typ obrázku: {result['image_type']}")
print(f"Navržená kategorie: {result['suggested_category']} ({result['category_confidence']:.1%})")

if result['recommendations']:
    print("\n📋 Doporučení:")
    for rec in result['recommendations']:
        print(f"  • {rec}")

## 8. Specializované modely pro konkrétní domény

In [ ]:
# Model pro klasifikaci jídla
food_classifier = pipeline(
    "image-classification",
    model="nateraw/food",
    device=0 if device == "cuda" else -1
)

print("✅ Food classifier načten")

In [ ]:
# Test na obrázku jídla
food_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Good_Food_Display_-_NCI_Visuals_Online.jpg/800px-Good_Food_Display_-_NCI_Visuals_Online.jpg"

try:
    food_image = load_image(food_url)
    food_results = food_classifier(food_image, top_k=5)
    
    print("Klasifikace jídla:")
    for r in food_results:
        print(f"  {r['label']}: {r['score']:.1%}")
    
    show_image_with_results(food_image, food_results, "Food Classification")
except Exception as e:
    print(f"Chyba: {e}")

In [ ]:
# Model pro detekci NSFW obsahu (bezpečnost)
nsfw_classifier = pipeline(
    "image-classification",
    model="Falconsai/nsfw_image_detection",
    device=0 if device == "cuda" else -1
)

def check_content_safety(image):
    """
    Kontrola bezpečnosti obsahu obrázku.
    """
    results = nsfw_classifier(image)
    
    # Najdi skóre pro bezpečný obsah
    safe_score = 0
    nsfw_score = 0
    
    for r in results:
        label = r['label'].lower()
        if 'normal' in label or 'safe' in label or 'sfw' in label:
            safe_score = r['score']
        elif 'nsfw' in label or 'explicit' in label:
            nsfw_score = r['score']
    
    is_safe = safe_score > nsfw_score
    
    return {
        "is_safe": is_safe,
        "safe_score": safe_score,
        "nsfw_score": nsfw_score,
        "recommendation": "OK" if is_safe else "Obrázek vyžaduje ruční kontrolu"
    }

# Test
image = load_image(test_url)
safety = check_content_safety(image)

print("Kontrola bezpečnosti obsahu:")
print(f"  Bezpečný: {'✅ Ano' if safety['is_safe'] else '⚠️ Vyžaduje kontrolu'}")
print(f"  Safe score: {safety['safe_score']:.1%}")
print(f"  Doporučení: {safety['recommendation']}")

## 9. Kompletní workflow pro automatizaci

In [ ]:
class CompleteImageWorkflow:
    """
    Kompletní workflow pro zpracování obrázků v produkčním prostředí.
    """
    
    def __init__(self, categories: dict):
        self.processor = EshopImageProcessor(categories)
        self.nsfw_classifier = nsfw_classifier
        
    def full_analysis(self, image_source, product_info=None):
        """
        Kompletní analýza obrázku.
        """
        try:
            image = load_image(image_source)
        except Exception as e:
            return {"status": "error", "error": str(e)}
        
        # 1. Bezpečnost
        safety = check_content_safety(image)
        if not safety['is_safe']:
            return {
                "status": "rejected",
                "reason": "Obsah neprošel kontrolou bezpečnosti",
                "safety_details": safety
            }
        
        # 2. Zpracování
        result = self.processor.process_product_image(
            image_source, 
            product_info.get('name') if product_info else None
        )
        
        # 3. Finální verdikt
        if result['status'] == 'error':
            return result
        
        result['safety_check'] = 'passed'
        result['ready_for_publish'] = (
            result['is_suitable'] and 
            safety['is_safe']
        )
        
        return result
    
    def generate_report(self, results):
        """
        Generuje souhrnný report.
        """
        total = len(results)
        ready = sum(1 for r in results if r.get('ready_for_publish', False))
        rejected = sum(1 for r in results if r.get('status') == 'rejected')
        errors = sum(1 for r in results if r.get('status') == 'error')
        needs_review = total - ready - rejected - errors
        
        report = f"""
╔══════════════════════════════════════════════════╗
║         REPORT ZPRACOVÁNÍ OBRÁZKŮ               ║
╠══════════════════════════════════════════════════╣
║  Celkem zpracováno:    {total:>5}                    ║
║  ✅ Připraveno k pub.: {ready:>5}                    ║
║  ⚠️ Vyžaduje úpravu:   {needs_review:>5}                    ║
║  ❌ Zamítnuto:         {rejected:>5}                    ║
║  🔴 Chyby:             {errors:>5}                    ║
╚══════════════════════════════════════════════════╝
        """
        return report

# Test
workflow = CompleteImageWorkflow(eshop_categories)
result = workflow.full_analysis(test_url, {"name": "Test produkt"})

print("Kompletní analýza:")
print(f"  Ready for publish: {'✅' if result.get('ready_for_publish') else '❌'}")
print(f"  Safety check: {result.get('safety_check', 'N/A')}")
print(f"  Quality: {result.get('quality', {}).get('rating', 'N/A')}")

## Shrnutí

V tomto notebooku jsme se naučili:

1. **Vision Transformer (ViT)** - základní klasifikace obrázků pomocí ImageNet kategorií
2. **CLIP Zero-Shot** - klasifikace podle libovolných textových popisů bez trénování
3. **Kontrola kvality** - automatické vyhodnocení kvality produktových fotografií
4. **Třídění produktů** - automatická kategorizace produktových obrázků
5. **Detekce typu** - rozlišení fotografií, ilustrací, screenshotů
6. **Bezpečnostní kontrola** - detekce nevhodného obsahu
7. **Kompletní workflow** - produkční pipeline pro e-shop

### Další kroky
- Notebook 9: Object Detection a Instance Segmentation